This notebook was prepared by [Donne Martin](https://github.com/donnemartin). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Challenge Notebook

## Problem: Find the shortest path between two nodes in a graph.

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)
* [Solution Notebook](#Solution-Notebook)

## Constraints

* Is the graph directed?
    * Yes
* Is the graph weighted?
    * No
* Can we assume we already have Graph and Node classes?
    * Yes
* Are the inputs two Nodes?
    * Yes
* Is the output a list of Node keys that make up the shortest path?
    * Yes
* If there is no path, should we return None?
    * Yes
* Can we assume this is a connected graph?
    * Yes
* Can we assume the inputs are valid?
    * Yes
* Can we assume this fits memory?
    * Yes

## Test Cases

Input:
* `add_edge(source, destination, weight)`

```
graph.add_edge(0, 1)
graph.add_edge(0, 4)
graph.add_edge(0, 5)
graph.add_edge(1, 3)
graph.add_edge(1, 4)
graph.add_edge(2, 1)
graph.add_edge(3, 2)
graph.add_edge(3, 4)
```

Result:
* search_path(start=0, end=2) -> [0, 1, 3, 2]
* search_path(start=0, end=0) -> [0]
* search_path(start=4, end=5) -> None

## Algorithm

Refer to the [Solution Notebook](http://nbviewer.ipython.org/github/donnemartin/interactive-coding-challenges/blob/master/graphs_trees/graph_path_exists/path_exists_solution.ipynb).  If you are stuck and need a hint, the solution notebook's algorithm discussion might be a good place to start.

## Code

In [55]:
# %load ../graph/graph.py
from enum import Enum  # Python 2 users: Run pip install enum34


class State(Enum):
    unvisited = 0
    visiting = 1
    visited = 2


class Node:

    def __init__(self, key):
        self.key = key
        self.visit_state = State.unvisited
        self.incoming_edges = 0
        self.adj_nodes = {}  # Key = key, val = Node
        self.adj_weights = {}  # Key = key, val = weight

    def __repr__(self):
        return str(self.key)

    def __lt__(self, other):
        return self.key < other.key

    def add_neighbor(self, neighbor, weight=0):
        if neighbor is None or weight is None:
            raise TypeError('neighbor or weight cannot be None')
        neighbor.incoming_edges += 1
        self.adj_weights[neighbor.key] = weight
        self.adj_nodes[neighbor.key] = neighbor

    def remove_neighbor(self, neighbor):
        if neighbor is None:
            raise TypeError('neighbor cannot be None')
        if neighbor.key not in self.adj_nodes:
            raise KeyError('neighbor not found')
        neighbor.incoming_edges -= 1
        del self.adj_weights[neighbor.key]
        del self.adj_nodes[neighbor.key]


class Graph:

    def __init__(self):
        self.nodes = {}  # Key = key, val = Node

    def add_node(self, key):
        if key is None:
            raise TypeError('key cannot be None')
        if key not in self.nodes:
            self.nodes[key] = Node(key)
        return self.nodes[key]

    def add_edge(self, source_key, dest_key, weight=0):
        if source_key is None or dest_key is None:
            raise KeyError('Invalid key')
        if source_key not in self.nodes:
            self.add_node(source_key)
        if dest_key not in self.nodes:
            self.add_node(dest_key)
        self.nodes[source_key].add_neighbor(self.nodes[dest_key], weight)

    def add_undirected_edge(self, src_key, dst_key, weight=0):
        if src_key is None or dst_key is None:
            raise TypeError('key cannot be None')
        self.add_edge(src_key, dst_key, weight)
        self.add_edge(dst_key, src_key, weight)

In [56]:
from collections import deque

class GraphShortestPath(Graph):

    def shortest_path(self, source_key, dest_key):
        if (source_key == dest_key and self.nodes[source_key] is not None):
            return [source_key]
        
        queue = deque()
        edge = {}
        queue.append(source_key)
        while len(queue) > 0:
            node = self.nodes[queue.popleft()]
            node.visit_state = State.visited
            for adjNode in node.adj_nodes.values():
                if adjNode.visit_state == State.visited:
                    continue
                    
                edge[adjNode.key] = node
                
                if adjNode.key == dest_key:
                    break;
                
                queue.append(adjNode.key)
        
        node = edge[dest_key]
        
        if node is None:
            return None

        path = [dest_key]
        
        while node.key != source_key:
            path.append(node.key)
            node = edge[node.key]

        path.append(node.key)
        
        path.reverse()
        
        for node in self.nodes.values():
            node.visit_state = State.unvisited
        
        return path        

## Unit Test

**The following unit test is expected to fail until you solve the challenge.**

In [57]:
# %load test_shortest_path.py
from nose.tools import assert_equal


class TestShortestPath(object):

    def test_shortest_path(self):
        nodes = []
        graph = GraphShortestPath()
        for id in range(0, 6):
            nodes.append(graph.add_node(id))
        graph.add_edge(0, 1)
        graph.add_edge(0, 4)
        graph.add_edge(0, 5)
        graph.add_edge(1, 3)
        graph.add_edge(1, 4)
        graph.add_edge(2, 1)
        graph.add_edge(3, 2)
        graph.add_edge(3, 4)

        assert_equal(graph.shortest_path(nodes[0].key, nodes[2].key), [0, 1, 3, 2])
        print '0 -> 0'
        assert_equal(graph.shortest_path(nodes[0].key, nodes[0].key), [0])
        assert_equal(graph.shortest_path(nodes[4].key, nodes[5].key), None)

        print('Success: test_shortest_path')


def main():
    test = TestShortestPath()
    test.test_shortest_path()


if __name__ == '__main__':
    main()

0 -> 0


KeyError: 5

## Solution Notebook

Review the [Solution Notebook](http://nbviewer.ipython.org/github/donnemartin/interactive-coding-challenges/blob/master/graphs_trees/graph_path_exists/path_exists_solution.ipynb) for a discussion on algorithms and code solutions.